In [2]:
!pip install noisereduce

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import noisereduce as nr
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

In [47]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the path to your spectrogram images
train_directory = '/content/drive/MyDrive/P2_DeepLearning/birdclef-2023.zip (Unzipped Files)/random10_prep_data/train'
validation_directory = '/content/drive/MyDrive/P2_DeepLearning/birdclef-2023.zip (Unzipped Files)/random10_prep_data/test'

#train_directory = r'/content/drive/MyDrive/P2_DeepLearning/birdclef-2023.zip (Unzipped Files)/random10_prep_data/train'
#validation_directory = r'/content/drive/MyDrive/P2_DeepLearning/birdclef-2023.zip (Unzipped Files)/random10_prep_data/test'

#reserved_directory = '/content/drive/MyDrive/P2_DeepLearning/birdclef-2023.zip (Unzipped Files)/random10_prep_data/reserved'

# Image dimensions
#img_width, img_height = 884, 322
img_width, img_height = 322, 884
# Rescale the images by dividing every pixel in every image by 255
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
reserved_datagen = ImageDataGenerator(rescale=1./255)

# Load images from the directory and prepare them for training
train_generator = train_datagen.flow_from_directory(
    #train_directory,
    directory = train_directory,
    target_size=(img_height, img_width),
    color_mode = "grayscale",
    batch_size=16,
    class_mode='categorical',
    shuffle = True,
    seed = 91825)

validation_generator = validation_datagen.flow_from_directory(
    #validation_directory,
    directory = validation_directory,
    target_size=(img_height, img_width),
    color_mode = "grayscale",
    batch_size=16,
    class_mode='categorical',
    shuffle = True,
    seed = 18232)

#reserved_generator = reserved_datagen.flow_from_directory(
#    directory = reserved_directory,
#    target_size=(img_height, img_width),
#    color_mode = "grayscale",
#    batch_size=16,
#    class_mode='categorical',
#    shuffle = False)

Found 1836 images belonging to 10 classes.
Found 572 images belonging to 10 classes.


In [48]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
model = Sequential()

# Convolutional layer with 32 filters, a kernel size of 3x3, and ReLU activation
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(322, 884, 1)))
#model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(884, 322, 1)))
model.add(MaxPooling2D((2, 2)))

# Adding another convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(MaxPooling2D((2, 2)))

# Adding another convolutional layer
#model.add(Conv2D(128, (3, 3), activation='softmax'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Fully connected layer with 128 units and ReLU activation
model.add(Dense(128, activation='relu'))
#model.add(Dense(128, activation='softmax'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer with a unit for each class and softmax activation
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model summary
model.summary()


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 320, 882, 32)      320       
                                                                 
 max_pooling2d_24 (MaxPooli  (None, 160, 441, 32)      0         
 ng2D)                                                           
                                                                 
 conv2d_25 (Conv2D)          (None, 158, 439, 64)      18496     
                                                                 
 max_pooling2d_25 (MaxPooli  (None, 79, 219, 64)       0         
 ng2D)                                                           
                                                                 
 conv2d_26 (Conv2D)          (None, 77, 217, 128)      73856     
                                                                 
 max_pooling2d_26 (MaxPooli  (None, 38, 108, 128)     

In [49]:
# Train the model
history = model.fit(
    train_generator,
    #steps_per_epoch=train_generator.samples // 16,
    steps_per_epoch = train_generator.samples // 160,
    #epochs=5,
    epochs=1,
    validation_data=validation_generator,
    #validation_steps=validation_generator.samples // 16)
    validation_steps=validation_generator.samples // 160)

#history = model.fit( train_dataset, batch_size=100, epochs=50 )

11/11 [==============================] - 126s 11s/step - loss: 1.8613 - accuracy: 0.5227 - val_loss: 1.6773 - val_accuracy: 0.5417


In [50]:
test_directory = '/content/drive/MyDrive/P2_DeepLearning/birdclef-2023.zip (Unzipped Files)/random10_prep_data/test'

test_datagen = ImageDataGenerator(rescale=1./255)
#test_generator = test_datagen.flow_from_directory(
#    test_directory,
#    target_size=(img_height, img_width),
#    batch_size=16,
#    class_mode='categorical',
#    shuffle=False)  # It's important not to shuffle the test data

test_generator = test_datagen.flow_from_directory(
    #test_directory,
    directory = test_directory,
    target_size=(img_height, img_width),
    color_mode = "grayscale",
    batch_size=16,
    class_mode='categorical',
    shuffle = False)


Found 572 images belonging to 10 classes.


In [52]:
#test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // 16)
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // 16)
print("Test accuracy:", test_accuracy)
print("Test loss:", test_loss)


35/35 [==============================] - 106s 3s/step - loss: 1.6416 - accuracy: 0.5643
Test accuracy: 0.5642856955528259
Test loss: 1.6416442394256592


In [ ]:
# from sklearn.metrics import classification_report, confusion_matrix

# # Predict classes
# predictions = model.predict(test_generator, steps=test_generator.samples // 16)
# predicted_classes = np.argmax(predictions, axis=1)

# # Get true labels
# true_classes = test_generator.classes

# # Calculate classification report and confusion matrix
# report = classification_report(true_classes, predicted_classes, target_names=test_generator.class_indices.keys())
# conf_matrix = confusion_matrix(true_classes, predicted_classes)

# print(report)
# print(conf_matrix)
